### --- Day 13: Transparent Origami ---
You reach another volcanically active part of the cave. It would be nice if you could do some kind of thermal imaging so you could tell ahead of time which caves are too hot to safely enter.

Fortunately, the submarine seems to be equipped with a thermal camera! When you activate it, you are greeted with:

Congratulations on your purchase! To activate this infrared thermal imaging
camera system, please enter the code found on page 1 of the manual.
Apparently, the Elves have never used this feature. To your surprise, you manage to find the manual; as you go to open it, page 1 falls out. It's a large sheet of transparent paper! The transparent paper is marked with random dots and includes instructions on how to fold it up (your puzzle input). For example:

    6,10
    0,14
    9,10
    0,3
    10,4
    4,11
    6,0
    6,12
    4,1
    0,13
    10,12
    3,4
    3,0
    8,4
    1,10
    2,14
    8,10
    9,0

fold along y=7
fold along x=5
The first section is a list of dots on the transparent paper. 0,0 represents the top-left coordinate. The first value, x, increases to the right. The second value, y, increases downward. So, the coordinate 3,0 is to the right of 0,0, and the coordinate 0,7 is below 0,0. The coordinates in this example form the following pattern, where # is a dot on the paper and . is an empty, unmarked position:

    ...#..#..#.
    ....#......
    ...........
    #..........
    ...#....#.#
    ...........
    ...........
    ...........
    ...........
    ...........
    .#....#.##.
    ....#......
    ......#...#
    #..........
    #.#........
Then, there is a list of fold instructions. Each instruction indicates a line on the transparent paper and wants you to fold the paper up (for horizontal y=... lines) or left (for vertical x=... lines). In this example, the first fold instruction is fold along y=7, which designates the line formed by all of the positions where y is 7 (marked here with -):

    ...#..#..#.
    ....#......
    ...........
    #..........
    ...#....#.#
    ...........
    ...........
    -----------
    ...........
    ...........
    .#....#.##.
    ....#......
    ......#...#
    #..........
    #.#........
Because this is a horizontal line, fold the bottom half up. Some of the dots might end up overlapping after the fold is complete, but dots will never appear exactly on a fold line. The result of doing this fold looks like this:

    #.##..#..#.
    #...#......
    ......#...#
    #...#......
    .#.#..#.###
    ...........
    ...........
Now, only 17 dots are visible.

Notice, for example, the two dots in the bottom left corner before the transparent paper is folded; after the fold is complete, those dots appear in the top left corner (at 0,0 and 0,1). Because the paper is transparent, the dot just below them in the result (at 0,3) remains visible, as it can be seen through the transparent paper.

Also notice that some dots can end up overlapping; in this case, the dots merge together and become a single dot.

The second fold instruction is fold along x=5, which indicates this line:

    #.##.|#..#.
    #...#|.....
    .....|#...#
    #...#|.....
    .#.#.|#.###
    .....|.....
    .....|.....
Because this is a vertical line, fold left:

    #####
    #...#
    #...#
    #...#
    #####
    .....
    .....
The instructions made a square!

The transparent paper is pretty big, so for now, focus on just completing the first fold. After the first fold in the example above, 17 dots are visible - dots that end up overlapping after the fold is completed count as a single dot.

How many dots are visible after completing just the first fold instruction on your transparent paper?

In [37]:
import numpy as np
import re

def manual_input(input_file:str):
    """Build array from single column text file. Convert to integers"""
    coords = []
    instruc = []
    instruc_re = re.compile('(x|y)=\d*')
    
    with open(input_file, 'r') as file:
        for line in file:
            if line == '': pass
            elif bool(re.search(instruc_re, line)):
                info = line.strip().split(' ')[2]
                instruc.append([info.split('=')[0], int(info.split('=')[1])])
            elif len(line.split(',')) == 2:
                x, y = line.strip().split(',') 
                coords.append([int(x),int(y)])
                               
        return coords, instruc


def construct_matrix(coords:list) -> np.array:
    for coord in coords:
        list_x_y = list(zip(*coords))
        tuples_x_y = (list_x_y[1],list_x_y[0])
        max_x = max(tuples_x_y[0]) + 1
        max_y = max(tuples_x_y[1]) + 1
        page = np.zeros((max_x, max_y), dtype=np.bool)
        page[tuples_x_y] = 1
        return page
        
    
    
def fold_paper(array:np.array, axis=None, fold=None) -> np.array:
    """Fold an array along an axis"""
    if axis == 'x':
        left = array[::,:fold:1]
        right = array[::,:fold:-1]
        shape_dif =  left.shape[1] - right.shape[1]
        if shape_dif < 0:
            left = np.pad(left, [(0,0), (abs(shape_dif),0)], mode='constant', constant_values=False)       
        elif shape_dif > 0:
            right = np.pad(right, [(0,0), (shape_dif, 0)], mode='constant', constant_values=False)       
        return left + right
    
    elif axis == 'y':
        upper = array[:fold:1]
        lower = array[:fold:-1]
        shape_dif =  upper.shape[0] - lower.shape[0]
        if shape_dif > 0:
            lower = np.pad(lower, [(shape_dif,0), (0,0)], mode='constant', constant_values=False)       
        elif shape_dif < 0:
            upper = np.pad(upper, [(abs(shape_dif),0), (0,0)], mode='constant', constant_values=False)       
        return upper + lower
    
    else:
        print(axis, fold)
    

In [56]:
# Part1 SAMPLE

coords, instruc = manual_input('data/d13_manual_sample.txt')
page = construct_matrix(coords)

print(page.astype(int))
print(page.shape)

for i in instruc:
    page = fold_paper(page, i[0], i[1])
    print(f'Fold instructions: {i}')
    print(f'The current page size: {page.shape}')
    print(page.astype(int))
    print(np.sum(page))
print(f'The final page size {page.shape} and total dots: {np.sum(page)}')

[[0 0 0 1 0 0 1 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0]]
(15, 11)
Fold instructions: ['y', 7]
The current page size: (7, 11)
[[1 0 1 1 0 0 1 0 0 1 0]
 [1 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 1 0 0 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]
17
Fold instructions: ['x', 5]
The current page size: (7, 5)
[[1 1 1 1 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]
16
The final page size (7, 5) and total dots: 16


In [57]:
# Part1

coords, instruc = manual_input('data/d13_manual.txt')
page = construct_matrix(coords)

print(page.astype(int))
print(page.shape)

for i in instruc:
    page = fold_paper(page, i[0], i[1])
    print(f'Fold instructions: {i}')
    print(f'The current page size: {page.shape}')
    print(page.astype(int))
    print(np.sum(page))
print(f'The final page size {page.shape} and total dots: {np.sum(page)}')

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(894, 1311)
Fold instructions: ['x', 655]
The current page size: (894, 655)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
708
Fold instructions: ['y', 447]
The current page size: (447, 655)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
583
Fold instructions: ['x', 327]
The current page size: (447, 327)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
493
Fold instructions: ['y', 223]
The current page size: (223, 327)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
408
Fold instructions: ['x', 163]
The current page size: (223, 163)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 